## 룰 CSV 로드

In [2]:
import os
import pandas as pd

# -----------------------------------
# 1) 룰 CSV 로드
# -----------------------------------


BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
RULE_PATH = os.path.join(
    BASE_DIR,
    "..",
    "01_data_text_rule",
    "basic_data",
    "04_group_shape_design_rule.csv"
)

df_rule = pd.read_csv(RULE_PATH)

# len_wid만 있고 나머지 없는 경우 보정
if "nail_len_type" not in df_rule.columns and "len_wid" in df_rule.columns:
    df_rule[["nail_len_type", "nail_wid_type"]] = df_rule["len_wid"].str.split("-", expand=True)

# 디자인 확률 컬럼 이름 확인
if "prob_design_given_group_shape" not in df_rule.columns:
    # score_joint 기준으로 (len,wid,shape) 안에서 정규화
    df_rule["prob_design_given_group_shape"] = (
        df_rule.groupby(["nail_len_type", "nail_wid_type", "shape"])["score_joint"]
               .transform(lambda x: x / x.sum() if x.sum() > 0 else x)
    )

df_rule.head()

,Unnamed: 0,len_wid,nail_len_type,nail_wid_type,group_id,shape,design,score_joint,design_given_group_shape,prob_design_given_group_shape
0,0,long-narrow,long,narrow,L_N,square,art,0.039976,0.08125,0.08125
1,1,long-narrow,long,narrow,L_N,square,french,0.156830,0.31875,0.31875
2,2,long-narrow,long,narrow,L_N,square,glitter,0.083027,0.16875,0.16875
3,3,long-narrow,long,narrow,L_N,square,nudes,0.116854,0.23750,0.23750
4,4,long-narrow,long,narrow,L_N,square,ombre,0.095328,0.19375,0.19375


## 길이/너비 분류 함수

In [3]:
# -----------------------------------
# 2) 길이/너비 카테고리 분류 함수
# -----------------------------------

def classify_length_type(aspect_ratio: float) -> str:
    """
    aspect_ratio = 높이 / 너비 (세로가 길수록 값이 큼)

    예시 기준:
      - short :  aspect < 0.9      (가로가 상대적으로 긴 손톱)
      - mid   :  0.9 <= aspect < 1.3
      - long  :  aspect >= 1.3     (세로가 확실히 긴 손톱)
    """
    if aspect_ratio < 0.9:
        return "short"
    elif aspect_ratio < 1.3:
        return "mid"
    else:
        return "long"


def classify_width_type(width_ratio: float) -> str:
    """
    width_ratio = 손톱 너비의 상대적인 비율
    (예: 해당 손톱 너비 / 같은 손 안에서 가장 넓은 손톱 너비)

    예시 기준:
      - narrow :  width_ratio < 0.8
      - wide   :  width_ratio >= 0.8
    """
    if width_ratio < 0.8:
        return "narrow"
    else:
        return "wide"

## 길이/너비/shape → 디자인 TOP-K 추천

In [4]:
# -----------------------------------
# 3) len_type / wid_type / shape → 디자인 TOP-K 추천
# -----------------------------------

def recommend_designs(len_type: str,
                      wid_type: str,
                      shape_name: str,
                      df_rule: pd.DataFrame,
                      topk: int = 3) -> pd.DataFrame | None:
    """
    len_type: 'short' / 'mid' / 'long'
    wid_type: 'narrow' / 'wide'
    shape_name: 'square' / 'round' / 'oval' / 'almond' / 'stiletto'
    """
    mask = (
        (df_rule["nail_len_type"] == len_type) &
        (df_rule["nail_wid_type"] == wid_type) &
        (df_rule["shape"] == shape_name)
    )
    sub = df_rule[mask].copy()
    if sub.empty:
        return None

    sub = sub.sort_values("prob_design_given_group_shape", ascending=False)
    return sub[["design", "prob_design_given_group_shape"]].head(topk)

def recommend_shape_and_designs(len_type: str,
                                wid_type: str,
                                df_rule: pd.DataFrame,
                                topk_design: int = 3):
    """
    len_type, wid_type만으로
      - shape 1개 추천
      - 해당 shape에서 디자인 TOP-K 추천
    """
    sub = df_rule[
        (df_rule["nail_len_type"] == len_type) &
        (df_rule["nail_wid_type"] == wid_type)
    ].copy()
    if sub.empty:
        return None, None

    # shape별로 전체 확률 합산 → 우선순위
    shape_score = (
        sub.groupby("shape")["prob_design_given_group_shape"]
           .sum()
           .sort_values(ascending=False)
    )

    best_shape = shape_score.index[0]

    # 그 shape 안에서 디자인 TOP-K
    sub_shape = sub[sub["shape"] == best_shape].copy()
    sub_shape = sub_shape.sort_values("prob_design_given_group_shape", ascending=False)

    top_designs = sub_shape[["design", "prob_design_given_group_shape"]].head(topk_design)
    return best_shape, top_designs


In [5]:
# -----------------------------------
# 4) 데모: 손톱 1개에 대해 추천
# -----------------------------------

# 예시 값 (나중에 실제 측정값으로 대체)
height = 130   # px
width  = 80    # px
aspect_ratio = height / width         # 길이 비율
width_ratio  = 0.9                    # 예: w / max_w_in_hand

len_type = classify_length_type(aspect_ratio)
wid_type = classify_width_type(width_ratio)

print("길이 타입:", len_type)
print("너비 타입:", wid_type)

best_shape, top_designs = recommend_shape_and_designs(len_type, wid_type, df_rule)

print("\n추천 쉐입:", best_shape)
print("\n추천 디자인 TOP3:")
print(top_designs)

길이 타입: long
너비 타입: wide

추천 쉐입: almond

추천 디자인 TOP3:
     design  prob_design_given_group_shape
42  glitter                       0.333333
40      art                       0.296296
44    ombre                       0.277778
